In [1]:
#| default_exp interface_trimesh

## `trimesh` interface

> Convert `ObjMesh` to `trimesh`'s mesh class.


In this notebook, we define functions to convert our `ObjMesh` class to and from `trimesh` (https://trimesh.org/) which is a python library for triangle meshes. I generally prefer to use `igl`, but maybe `trimesh` has some feature you want.

Note: `trimesh` represents triangular meshes only, and its way of representing UV information is not ideal. It is not recommended to edit mesh topology in `trimesh` if the UV mapping has already been defined.

In [2]:
#| export

from blender_tissue_cartography import mesh as tcmesh

import numpy as np
import trimesh

In [4]:
#| hide
np.set_printoptions(suppress=True)

In [6]:
#| export

def convert_to_trimesh(mesh: tcmesh.ObjMesh, add_texture_info=None) -> trimesh.Trimesh:
    """
    Convert tcmesh.ObjMesh to trimesh.Trimesh
    
    See https://trimesh.org/trimesh.base.html
    Note: normal information is recalculated. Discards any non-triangle faces.
    
    Texture is saved as a vertex attribute via v_tex_coords_matrix. Note that this discards
    information since a vertex can have multiple texture coordinates!
    For this reason, we also add the texture coordinates as a (n_faces, 3, s)-array attribute
    `face_tex`. Note: this will _not_ be updated if you remesh etc.
    
    Parameters
    ----------
    mesh : tcmesh.ObjMesh
    add_texture_info : None or bool
        Whether to add texture info to the trimesh.Trimesh. If None, texture is added if
        available for at least one vertex.
    Returns
    -------
    trimesh.Trimesh

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular. discarding non-triangular faces")
    add_texture_info = ((not mesh.only_vertices and len(mesh.texture_vertices) > 0)
                        if add_texture_info is None else add_texture_info)
    if not add_texture_info:
        return trimesh.Trimesh(mesh.vertices, mesh.tris)
    texture = trimesh.visual.texture.TextureVisuals(uv=mesh.vertex_textures)
    converted = trimesh.Trimesh(mesh.vertices, mesh.tris, visual=texture)
    converted.face_tex = tcmesh.index_else_nan(mesh.texture_vertices, mesh.texture_tris)
    return converted

In [7]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference.obj"
mesh_data = tcmesh.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcmesh.ObjMesh.read_obj(mesh_fname_ref)

  o embryo_rect


In [10]:
trimesh_data = convert_to_trimesh(mesh_data)
np.allclose(mesh_data.vertices, trimesh_data.vertices)

True

In [12]:
trimesh_normals = trimesh_data.vertex_normals
trimesh_normals = (trimesh_normals.T / np.linalg.norm(trimesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_data.normals, trimesh_normals)

array([6.27953731, 6.26481851, 6.27572446, ..., 6.24734426, 6.22183872,
       6.25817202])

In [14]:
#| export

def convert_from_trimesh(mesh: trimesh.Trimesh, reconstruct_texture_from_faces=True,
                           texture_vertex_decimals=10) -> tcmesh.ObjMesh:
    """
    Convert trimesh mesh to ObjMesh.
    
    Texture vertices can be reconstructed from face attribute face_tex or from 
    vertex attribute vertex_tex_coord_matrix. Reconstruction from face texture can accommodate
    multiple texture coordinates per vertex (e.g. for UV maps with seams).
    
    Texture vertices are rounded to texture_vertex_decimals decimals
    """
    vertices = mesh.vertices
    normals = mesh.vertex_normals
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if not hasattr(mesh.visual, 'uv'):
        faces = [[2*[v,] for v in f] for f in mesh.faces]
        return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals)
    if hasattr(mesh.visual, 'uv') and not reconstruct_texture_from_faces:
        faces = [[2*[v,] for v in f] for f in mesh.faces]
        return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=mesh.visual.uv)
    # reconstruct texture vertices - big pain.
    texture_vertices = np.vstack([mesh.face_tex[:,i,:] for i in [0,1,2]])
    texture_vertices = np.round(texture_vertices, decimals=texture_vertex_decimals)
    texture_vertices_unique, inverse_index = np.unique(texture_vertices, axis=0, return_inverse=True)

    n_faces = mesh.faces.shape[0]
    faces = [[[v, inverse_index[ifc+iv*n_faces]] for iv, v in enumerate(fc)]
              for ifc, fc in enumerate(mesh.faces)]

    return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=texture_vertices_unique)

In [15]:
trimesh_ref = convert_to_trimesh(mesh_ref)

In [16]:
mesh_seams = tcmesh.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")
trimesh_seams = convert_to_trimesh(mesh_seams,add_texture_info=True)

  o Drosophila_CAAX-mCherry_mesh_remeshed
  l 2534 8160
/home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/trimesh/grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)


In [18]:
convert_from_trimesh(trimesh_seams, reconstruct_texture_from_faces=False).texture_vertices.shape, mesh_seams.texture_vertices.shape

((8159, 2), (8288, 2))